In [6]:
api_key = 'RGAPI-3f1c31aa-c78c-4588-8f69-054e020bf77b'

In [32]:
import requests
import os
from psycopg2 import pool
from dotenv import load_dotenv
from psycopg2.extras import Json, DictCursor

In [8]:
r = requests.get("https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/clu/clu1?api_key=" + api_key)
print(r.json())

{'puuid': 'R8_dzuvxZm0VsqU-paXM3wijLavgPmiZDmKIHFm8HddnUGvsgHTlCQuWdlHNRJlBIaEFyZYD87Avug', 'gameName': 'clu', 'tagLine': 'clu1'}


In [9]:
# setsuko, soju, dishsoap
streamers = ['e62ZthNQBc1bjVfXrjDvNIQV-0Bd7GUeaRv_fiZqx5nc5OXaY6OAmM4wav043AJgxKQShtA1s9IIOg', 'hVi-4fc19ymaRf4HvUmHNkZCoALJPUXgGKcKGgBlLZTqYZRVtQwglGQCIBj0jou0wA35CcH8sdeEgw', 'NUnja2Bl0CPRiimqr1PesnOnDsxXa6eAg5xRhNVBdxgAzBopnpcWNTZZ9r9YjNhEtf5tx2DkvXdJ9Q']

In [10]:
# for streamer_puuid in streamers:
#   matches = requests.get("https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + streamer_puuid + "&api_key=" + api_key)


In [31]:


# Load .env file
load_dotenv()

# Get the connection string from the environment variable
connection_string = os.getenv('DATABASE_URL')

# Create a connection pool
connection_pool = pool.SimpleConnectionPool(
    1,  # Minimum number of connections in the pool
    10,  # Maximum number of connections in the pool
    connection_string
)

# Check if the pool was created successfully
if connection_pool:
    print("Connection pool created successfully")

# Get a connection from the pool
conn = connection_pool.getconn()

# Create a cursor object
cur = conn.cursor()

# Execute SQL commands to retrieve the current time and version from PostgreSQL
cur.execute('SELECT NOW();')
time = cur.fetchone()[0]

cur.execute('SELECT version();')
version = cur.fetchone()[0]

# Close the cursor and return the connection to the pool
cur.close()
connection_pool.putconn(conn)

# Close all connections in the pool
connection_pool.closeall()

# Print the results
print('Current time:', time)
print('PostgreSQL version:', version)

Connection pool created successfully
Current time: 2024-11-06 19:30:33.550193+00:00
PostgreSQL version: PostgreSQL 16.4 on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit


In [33]:
current_puuid = streamers[0]

# get recent match data
matches = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/' + current_puuid + '/ids?start=0&count=20&api_key=' + api_key).json()

# check that matches are not already in db
# later :D:D:D

# get the stuff
# define carries and tier
carries = []
match_dict_array = []
for match in matches:
  match_dict = {}
  match_dict['match_id'] = match
  match_data = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/' + match + '?api_key=' + api_key).json()


  #get metadata
  match_dict['patch'] = match_data['info']['game_version'][-6:-1]
  match_dict['game_datetime'] = match_data['info']['game_datetime']

  game_information = match_data['info']['participants']
  # print(game_information[0])
  for board in game_information:
    curr_dict = {}
    curr_dict['augments'] = board['augments']
    curr_dict['level'] = board['level']
    curr_dict['placement'] = board['placement']
    curr_dict['traits'] = board['traits']
    curr_dict['units'] = board['units']
    curr_dict['puuid'] = board['puuid']
    curr_dict['gold_left'] = board['gold_left']

    r = requests.get('https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/' + curr_dict['puuid'] + '?api_key=' + api_key).json()
    curr_dict['username_tagline']= r['gameName'] + ' #' + r["tagLine"]

    match_dict[r['gameName'] + ' #' + r["tagLine"]] = curr_dict.copy()
  
    # figure out what comp they are playing

  # insert into match dict array
  match_dict_array.append(match_dict)
    





  break
print(match_dict_array)

[{'match_id': 'NA1_5138706817', 'patch': '14.20', 'game_datetime': 1729636996808, 'RamKev #NA1': {'augments': ['TFT9_Augment_ArmyBuilding', 'TFT12_Augment_VanguardCrest', 'TFT9_Augment_Commander_FinalAscension'], 'level': 9, 'placement': 5, 'traits': [{'name': 'TFT12_Arcana', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT12_Ascendant', 'num_units': 1, 'style': 5, 'tier_current': 1, 'tier_total': 1}, {'name': 'TFT12_Blaster', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 3}, {'name': 'TFT12_Chrono', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT12_Eldritch', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT12_Frost', 'num_units': 3, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT12_Mage', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT12_Portal', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT12_Preserver

In [55]:
# Load .env file
load_dotenv()

# Get the connection string from the environment variable
connection_string = os.getenv('DATABASE_URL')

# Create a connection pool
connection_pool = pool.SimpleConnectionPool(
    1,  # Minimum number of connections in the pool
    10,  # Maximum number of connections in the pool
    connection_string
)

# Check if the pool was created successfully
if connection_pool:
    print("Connection pool created successfully")

# Get a connection from the pool
conn = connection_pool.getconn()

# Create a cursor object
cur = conn.cursor(cursor_factory=DictCursor)


for match in match_dict_array:
  cur.execute('INSERT into matches (match_id, patch, match_data) values (%s, %s, %s)',
    [match['match_id'], match['patch'], Json(match)])


conn.commit()

cur.close()
connection_pool.putconn(conn)

# Close all connections in the pool
connection_pool.closeall()

Connection pool created successfully


UniqueViolation: duplicate key value violates unique constraint "matches_pkey"
DETAIL:  Key (match_id)=(NA1_5138706817) already exists.
